# Part 1: Noise Model
---

A standard way to represent the noise in a quantum circuit is through Pauli operators (x, y, z). Build a function with input ,  and QuantumCircuit where:

- ```alpha```: Probability of having a random Pauli operator acting on the qubit after a one-qubit gate 
- ```beta```: Probability of having a random Pauli operator acting on the qubit after a two-qubit gate 
- ```QuantumCircuit```:  Quantum circuit where the noise will be added 

The output should be the Quantum Circuit with Noise 


I'm using pennylane to simulate the quantum circuits.

In [1]:
import pennylane as qml
from pennylane import numpy as np
from helper_func import statevector_to_braket, create_circuit_from_operations_measurements

The circuit is written as a list of operations along with their parameters and the wires they act on. The measurements from the cirucit is written as another list.

In [2]:
ops = [
    qml.Hadamard(wires=0),
    qml.CNOT(wires=[0, 1]),
    qml.RX(0.5, wires=3),
    qml.SWAP(wires=[1, 2])
]

measurements = [
    qml.state()
]

The quantum tape contains all the operations in a circuit including the measurements.

In [3]:
clean_tape = qml.tape.QuantumTape(ops, measurements)
clean_tape.circuit

[Hadamard(wires=[0]),
 CNOT(wires=[0, 1]),
 RX(0.5, wires=[3]),
 SWAP(wires=[1, 2]),
 state(wires=[])]

Simulate the circuit to obtain the statevector as an array. Convert the array to bra-ket notation for a better understanding.

Function for converting statevector from a complex array to bra-ket notation in helper_func

In [4]:
dev = qml.device("lightning.qubit", wires=4)
clean_sv = qml.execute([clean_tape], dev)[0]
print(clean_sv)
statevector_to_braket(qml.execute([clean_tape], dev)[0])

[0.68512454+0.j         0.        -0.17494102j 0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.68512454+0.j         0.        -0.17494102j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j        ]


'0.69+0.00j * |0000> + 0.00-0.17j * |0001> + 0.69+0.00j * |1010> + 0.00-0.17j * |1011>'

It is not mentioned if the error on two qubit gates occur on one or both the qubits. Therefore, I have assumed that the error only occurs on any one of the qubits.

In [5]:
# Function to make a noisy circuit from a noiseless circuit
# Randomly insert Pauli-X, Pauli-Y, or Pauli-Z gates after each gate in the circuit with input probabilities
def make_noisy(alpha, beta, ops):
    index = 0
    while index < len(ops):
        op = ops[index]
        num_wires = len(op.wires)
        op_wires = op.wires.tolist()
        if num_wires == 1 and np.random.rand() < alpha:
            new_op = np.random.choice([qml.PauliX, qml.PauliY, qml.PauliZ])(wires=op_wires)
            ops.insert(index + 1, new_op)
            index += 1
        elif num_wires == 2 and np.random.rand() < beta:
            new_wire = np.random.choice(op_wires)
            new_op = np.random.choice([qml.PauliX, qml.PauliY, qml.PauliZ])(wires=new_wire)
            ops.insert(index + 1, new_op)
            index += 1
        index += 1

In [6]:
alpha = 0.7
beta = 0.4

make_noisy(alpha, beta, ops)

We have the quantum circuit with noise.

In [7]:
noisy_tape = qml.tape.QuantumTape(ops, measurements)
noisy_tape.circuit

[Hadamard(wires=[0]),
 Y(0),
 CNOT(wires=[0, 1]),
 X(1),
 RX(0.5, wires=[3]),
 X(3),
 SWAP(wires=[1, 2]),
 Z(1),
 state(wires=[])]

Noisy statevector as an array and converted to bra-ket notation.

In [8]:
noisy_sv = qml.execute([noisy_tape], dev)[0]
print(noisy_sv)
statevector_to_braket(qml.execute([noisy_tape], dev)[0])

[ 0.        -0.j          0.        -0.j         -0.17494102-0.j
  0.        -0.68512454j -0.        +0.j         -0.        +0.j
 -0.        +0.j         -0.        +0.j          0.17494102+0.j
  0.        +0.68512454j  0.        +0.j          0.        +0.j
 -0.        -0.j         -0.        -0.j         -0.        -0.j
 -0.        -0.j        ]


'-0.17-0.00j * |0010> + 0.00-0.69j * |0011> + 0.17+0.00j * |1000> + 0.00+0.69j * |1001>'

The amount of noise in the circuit can be quantified by calculating the fidelity between the expected state and the noisy state. Fidelity provides a measure of how close the noisy state is to the intended result, with values ranging from 0 (orthogonal) to 1 (identical).

In [9]:
clean_dm = np.outer(clean_sv, np.conjugate(clean_sv))
noisy_dm = np.outer(noisy_sv, np.conjugate(noisy_sv))
fidelity = qml.math.fidelity(clean_dm, noisy_dm)
print(fidelity)

1.6483897267854733e-35


Proceed to Decompositions.ipynb